# AST 4930 Homework Assignment #4 (10 pts + 2 extra credits)

## All work is due Wednesday October 11 at 5 pm.

## Instruction: 
Do all homework in this Jupyter notebook and submit your final .ipynb file via Canvas. Show ALL your work and try to add comment lines as needed to describe what your code does. 

You are encouraged to discuss homework problems with your classmates. However, your python script and answers to the questions must be written by yourself.

It's fine if you use someone else's code you found on Internet, but make sure you write down the source. You don't have to cite me when you use anything from AST4930 Notebook.

It's a common sense but just to make sure, when you make a figure make sure they are readable/understandable. If you cannot see the title, labels, data points, or if you don't understand the point of the figure, I won't probably be able to see/understand them either.

## 1. Computational complexity (4 pts)

We learned that different machine learning algorithms have different computational costs. Let's check this out using the two moons dataset (see the cell below for an example). 

(a) While varying the number of data points in the training/test dataset from 100, 1000, 10000, to 100000, measure the time it takes for kNN, DT, and SVM to create a model (i.e., fit) and make predictions. For simplicity, instead of optimizing hyperparameters, use n_neighbors=10 for kNN, max_depth=4 for DT, and kernel="rbf", C=1, gamma=1 for SVM. (2 pt)

In [1]:
from sklearn import datasets

# This is the size of the data.
n = 100

# This creates a training dataset.
X_train, y_train = datasets.make_moons(n_samples=n, noise=0.3, random_state=0)

# This creates a test dataset for prediction. 
X_pred, y_pred = datasets.make_moons(n_samples=n, noise=0.3, random_state=1)

(b) Make a plot showing the running time vs. size of dataset for the three algorightms and for training/prediction -- you will have six curves in the end. Make sure your figure is readable. Discuss what you find from the plot. Using a logarithmic scale for both x and y axes will be helpful to interpret the result. (2 pt)

## 2. Support Vector Machine classifier (6+2 pts)

We will build Support Vector Machines to determine the hyperplane separating Super-Earths and Sub-Neptunes, the so-called radius valley in planet radius vs. orbital period plots (see Fulton et al. 2017; https://ui.adsabs.harvard.edu/abs/2017AJ....154..109F/abstract). To determine accurate planet parameters from transit surveys, accurate stellar parameters are required, because the transit depth only constrains $R_p/R_*$, where $R_p$ and $R_*$ are the planetary and stellar radius. In Van Eylen et al. (2018; https://ui.adsabs.harvard.edu/abs/2018MNRAS.479.4786V/abstract), authors used a sample of exoplanet host stars with parameters homogeneously measured from asteroseismology, which can provide highly precise masses and radii for a sample of bright stars, and determined that the radius valley can be describe by ${\rm log}_{10} R_p = m{\rm log}_{10} P + a$, where $m=-0.09^{+0.02}_{-0.04}$ and $a=0.37^{+0.04}_{-0.02}$.

In this homework problem, we will reproduce the SVM models of Van Eylen et al. (2018).

First things first: let's read in the data presented in Table 1 of Van Eylen et al. (2018). Here, I provide the data in a txt file (VanEylen.txt) but remember that in reality, making a ML model will always start with data acquisition and cleaning, which can be tedious! 

In [18]:
import numpy as np
import pandas as pd

data = np.loadtxt('./VanEylen.txt', 
               dtype={'names': ('name', 'Rp', 'Period', 'Mstar', 'Rstar'),
                     'formats': ('|U15',float,float,float,float)}
              )
data = pd.DataFrame(data)

In [19]:
data

,name,Rp,Period,Mstar,Rstar
0,Kepler-10b,1.473,0.837,0.920,1.0662
1,Kepler-10c,2.323,45.294,0.920,1.0662
2,Kepler-23b,1.694,7.107,1.078,1.5480
3,Kepler-23c,3.120,10.742,1.078,1.5480
4,Kepler-23d,2.235,15.274,1.078,1.5480
...,...,...,...,...,...
112,K02706.01,1.797,3.098,1.260,1.8600
113,Kepler-1392,0.684,2.128,0.990,1.3000
114,K02801.01,0.870,6.992,1.120,1.4500
115,Kepler-1394,1.040,3.938,1.510,1.9800


(a) Let's make a plot showing planet radius vs. orbital period, similar to Figure 2 of Van Eylen et al. (2018) but without additional data from Fulton et al. (2017). Similar to Figure 2 of Van Eylen et al. (2018), make two panels, one showing the entire data points and the other showing 1-100 days & 1-4 Earth radii to more clearly identify the radius valley. (1 pt)

(b) We would like to fit the radius valley using the SVM. However, in order to use the SVM, we need a labelled data in the first place. Luckily, we have a well separated dataset. In Van Eylen et al. (2018), they took a few different approaches to fit the radius valley. Here, let's use one of the fits from bootstrapping, that is, if a planet's radius and orbital periods satisfy ${\rm log}_{10} R_p > m{\rm log}_{10} P + a$, where $m=-0.10$ and $a=0.38$, then we will label them as sub-Neptunes. If ${\rm log}_{10} R_p \leq m{\rm log}_{10} P + a$, where $m=-0.10$ and $a=0.38$, we will label them as super-Earth. Using this criterion, make an array that contains the label (e.g., 0 if sub-Neptune, 1 if super-Earth).

Then, repeat 2(a) and make similar plots, but this time make the data points color-coded by their labels. Also, add a line showing ${\rm log}_{10} R_p = m{\rm log}_{10} P + a$, where $m=-0.10$ and $a=0.38$. Your plot should look similar to Figure 5 of Van Eylen et al. (2018), except that you will have a single line dividing super-Earths and sub-Neptunes instead of multiple lines representing 20 different fits. (1 pt)

(c) Now that we have a labelled dataset, we are ready to build SVM models. Using orbital period and planet radius (both in log scale) as two features, make a few linear SVM models varying the panelty parameter C from 0.01, 0.1, 1, 10, 100, to 1000. Then, make six figures showing the data, hyperplane, and support vectors. Don't forget that SVMs need feature scaling. (2 pt)

(d) Which penalty parameter $C$ do you think best seperates super-Earths and sub-Neptunes? And why? Write down an equation for the hyperplane in the form of ${\rm log}_{10} R_p = m{\rm log}_{10} P + a$. (2 pt)

(e) This is for extra credits.

Uncertainties on the slope $m$ and offset $a$ can be determined using bootstrapped samples ([random sampling with replacement](https://en.wikipedia.org/wiki/Bootstrapping_(statistics) )). Following Van Eylen et al. (2018), generate 1000 bootstrapped samples each of which has the same size as the original dataset. For each bootstrapped sample, compute the slope $m$ and offset $a$ of the hyperplane using the best $C$ you determined from 2(d). Compute the mean and standard deviation of $m$ and $a$. What is your final hyperplane, and how does that compare with the hyperplane Van Eylen et al. (2018) determined? (2 pt)  

Hint: For bootstrapping you can use [np.random.choice](https://numpy.org/doc/stable/reference/random/generated/numpy.random.choice.html).

### Congratulations! With this homework, you essentially reproduced results presented in a journal paper. If you enjoyed the homework and would like to do some additional related analysis, I recommend you have a look at the following papers. Reproducing (and extending) the following papers would be a good term project.

https://ui.adsabs.harvard.edu/abs/2021AJ....161..265D/abstract

https://ui.adsabs.harvard.edu/abs/2023MNRAS.519.4056H/abstract